In [ ]:
# imports 
import tkinter as tk
from tkinter import filedialog
import datetime
import json
import csv
import os

class MainWindow(tk.Tk):
    def __init__(self):
        super().__init__()
        self.geometry("1000x800")
        self.title('Notebook')
        self.notebook = []
        self.notes = []

        # Set the main frame's background to light blue
        self.frame_main = tk.Frame(self, bg='light blue')
        self.frame_main.pack(fill=tk.BOTH, expand=True)

        self.frame_notes = tk.Frame(self.frame_main, bg='light blue')  # Notes frame also with light blue background
        self.frame_notes.grid(row=2, column=3, rowspan=6, sticky='w')

        # Scorll bar 
        self.canvas_notes = tk.Canvas(self.frame_notes, bg='light blue')
        self.scrollbar_notes = tk.Scrollbar(self.frame_notes, orient="vertical", command=self.canvas_notes.yview)
        self.canvas_notes.configure(yscrollcommand=self.scrollbar_notes.set)

        self.frame_note_container = tk.Frame(self.canvas_notes, bg='light blue')
        self.canvas_notes.create_window((0, 0), window=self.frame_note_container, anchor='nw')

        self.scrollbar_notes.pack(side="right", fill="y")
        self.canvas_notes.pack(side="left", fill="both", expand=True)

        # Bind the configuration of the container to update the scroll region
        self.frame_note_container.bind("<Configure>", lambda e: self.canvas_notes.configure(scrollregion=self.canvas_notes.bbox("all")))
        self.search_var = tk.StringVar()
        self.entry_search = tk.Entry(self.frame_main, textvariable=self.search_var, width=40, bg='light blue', fg='#255075')
        self.entry_search.grid(padx=10, pady=10, row=0, column=1, sticky='w')

        # Create buttons with specified background and text color                
        btn01 = tk.Button(self.frame_main, text='Create New Note', bg='light blue', fg='#255075', command=self.new_note)
        btn01.grid(padx=10, pady=10, row=1, column=1)
        btn02 = tk.Button(self.frame_main, text='Open Notebook', bg='light blue', fg='#255075', command=self.open_notebook)
        btn02.grid(padx=10, pady=10, row=1, column=2)
        btn03 = tk.Button(self.frame_main, text='Save Notebook', bg='light blue', fg='#255075', command=self.save_notebook)
        btn03.grid(padx=10, pady=10, row=1, column=3)
        btn04 = tk.Button(self.frame_main, text='Quit', bg='light blue', fg='#255075', command=self.destroy)
        btn04.grid(padx=10, pady=10, row=1, column=4)
        btn_search = tk.Button(self.frame_main, text='Search', bg='light blue', fg='#255075', command=self.search_notes)
        btn_search.grid(padx=10, pady=10, row=0, column=2)
    
    def search_notes(self):
        search_query = self.search_var.get().lower()
        self.clear_frame(self.frame_notes)
        self.notes = []  # Reset the notes display list
        for note in self.notebook:
            if search_query in note['title'].lower() or search_query in note['text'].lower() or search_query in note['tags'].lower():
                filtered_note = MakeNote(master=self.frame_notes, note_dict=note, notebook=self.notebook)
                self.notes.append(filtered_note)
                filtered_note.pack(in_=self.frame_note_container, padx=10, pady=10)
                filtered_note.config(height=4, width=60, wraplength=300, justify=tk.LEFT)

    def new_note(self):  # Opens a new note form
        note_form = NoteForm(self, self.notebook, self.notes)
        return None

    def clear_frame(self, target_frame):  # Method for clearing old content from the frame
        for widgets in target_frame.winfo_children():
            widgets.destroy()

    def show_notes(self):  # Generates note objects and displays them in the main window
        self.clear_frame(self.frame_notes)  # Clears any previous display
        self.notes = []  # Resets the notes object list
        for note in self.notebook:  # Create new note objects from the notebook and display them
            new_note = MakeNote(master=self.frame_notes, note_dict=note, notebook=self.notebook)
            self.notes.append(new_note)
            new_note.pack(padx=10, pady=10)
            new_note.config(height=4, width=60, wraplength=300, justify=tk.LEFT)
        return None

    def open_notebook(self):
        filepath = filedialog.askopenfilename(filetypes=[("JSON files", "*.json"), 
                                                        ("CSV files", "*.csv"),
                                                        ("Text files", "*.txt"),
                                                        ("All files", "*.*")])
        if not filepath:
            return  # No file was selected

        extension = os.path.splitext(filepath)[1]
        with open(filepath, 'r') as file:
            if extension == ".json":
                self.notebook = json.load(file)
            elif extension == ".csv":
                reader = csv.DictReader(file)
                self.notebook = [row for row in reader]
            else:  # Treat any other file as a plain text file
                self.notebook = [{'title': 'Text File', 'text': file.read()}]  # Simplified representation

        self.show_notes()  # Display notes from the loaded file

    
    def save_notebook(self):
        # Specifying the directory
        save_directory = "/Users/Allison/Desktop/inst326/"
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)  # Creates the directory if it does not exist

        filetypes = [("JSON file", "*.json"), ("CSV file", "*.csv"), ("Text file", "*.txt")]
        file = filedialog.asksaveasfile(mode='w', initialdir=save_directory, defaultextension=filetypes, filetypes=filetypes)
        if file:
            ext = os.path.splitext(file.name)[1]
            if ext == ".json":
                json.dump(self.notebook, file, indent=4)
            elif ext == ".csv":
                keys = self.notebook[0].keys() if self.notebook else []
                writer = csv.DictWriter(file, fieldnames=keys)
                writer.writeheader()
                writer.writerows(self.notebook)
            elif ext == ".txt":
                for note in self.notebook:
                    file.write(f"{note}\n")  # Simple text representation of the dictionary
            file.close()
            self.show_notes()


# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example

# PROJECT 03 MODIFICATIONS
# ADD NOTE_ID, AUTHOR, AND SNIPPET FIELDS TO NoteForm

class NoteForm(tk.Toplevel):
    
    def __init__(self, master, notebook, notes): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window
        # set the new window's default parameters
        self.geometry("900x800") 
        self.title('New Note')
        
        # create a frame in the new window that covers the entire window
        self.frame_main = tk.Frame(self)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light blue')

        
        #define self.notebook as the notebook passed from the main window
        self.notebook = notebook
        #self.notes = notes (not necessary)
        
        # PROJECT 03 EDIT: ADD note_id
        self.last_id = len(self.notebook)
        

        
        #define default dummy text (for development purposes only)
        default_note = {"title":"new note title",
                     "text":"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Etiam sit amet suscipit mi, non porttitor mauris. Aliquam in lorem risus. Proin mauris mauris, varius ac vulputate sed, tempor nec lacus. Morbi sodales turpis in placerat semper. Donec bibendum blandit ante sit amet hendrerit.", 
                    "link":"If there is a link with this note enter it here.",
                    "tags":"enter hashtags here",
                    "id":self.last_id + 1,
                    "author":"Scott Dempwolf",
                    "snippet":"# enter executable code snippet here \nprint('Hello World')",
                    "meta":"metadata added at submission"}
                    
        note = default_note # provided in anticipation of note editing functionality
        
        
        # create some labels and put them in the grid
        # we are using the grid layout. Notice the sticky='e' attribute. 
        # this causes the label to 'stick' to the 'east' side of the grid cell
        title_label = tk.Label(self.frame_main, bg='light blue', text='Note Title:',fg='#255075')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')

        text_label = tk.Label(self.frame_main, bg='light blue', text='Note Text:',fg='#255075')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')

        link_label = tk.Label(self.frame_main, bg='light blue', text='Note Link:',fg='#255075')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')

        tag_label = tk.Label(self.frame_main, bg='light blue', text='Note Tags:',fg='#255075')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')
        
        tag_label = tk.Label(self.frame_main, bg='light blue', text='Note ID:', fg='#255075')
        tag_label.grid(padx=10, pady=10, row=5, column=0, sticky='e')
        
        tag_label = tk.Label(self.frame_main, bg='light blue', text='Note Author:',fg='#255075')
        tag_label.grid(padx=10, pady=10, row=6, column=0, sticky='e')
        
        tag_label = tk.Label(self.frame_main, bg='light blue', text='Snippet:', fg='#255075')
        tag_label.grid(padx=10, pady=10, row=7, column=0, sticky='e')

        # create our note title entry field
        self.note_title = tk.Entry(self.frame_main, width=80)
        self.note_title.grid(padx=10, pady=10, row=1, column=1, sticky='w')
#         self.note_title.insert(0, note["title"]) # adds default text (useful during development)

        # create our note text field
        self.note_text = tk.Text(self.frame_main, height=10, width=60)
        self.note_text.grid(padx=10, pady=10, row=2, column=1)
        self.note_text.insert('1.0', note["text"]) # adds default text (useful during development)

        # create our note link entry field
        self.note_link = tk.Entry(self.frame_main, width=80)
        self.note_link.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        self.note_link.insert(0, note["link"]) # adds default text (useful during development)

        # create our note tags entry field
        self.note_tags = tk.Entry(self.frame_main, width=80)
        self.note_tags.grid(padx=10, pady=10, row=4, column=1, sticky='w')
        self.note_tags.insert(0, note["tags"]) # adds default text (useful during development)

        # create our note id entry field
        self.note_id = tk.Entry(self.frame_main, width=80)
        self.note_id.grid(padx=10, pady=10, row=5, column=1, sticky='w')
        self.note_id.insert(0, note["id"]) # adds default text (useful during development)

        # create our note author entry field
        self.note_author = tk.Entry(self.frame_main, width=80)
        self.note_author.grid(padx=10, pady=10, row=6, column=1, sticky='w')
        self.note_author.insert(0, note["author"]) # adds default text (useful during development)

        # create our note snippet field
        self.snippet = tk.Text(self.frame_main, height=10, width=60)
        self.snippet.grid(padx=10, pady=10, row=7, column=1)
        self.snippet.insert('1.0', note["snippet"]) # adds default text (useful during development)        
        
        # create our note meta field if you want to add edit functionality
#         self.note_meta = tk.Entry(self.frame_main, width=80)
#         self.note_meta.grid(padx=10, pady=10, row=5, column=1, sticky='w')
#         self.note_meta.insert(0, note["meta"]) # adds default text (useful during development)
        

        # note that the parameters for the Entry box and Text box are slightly different.
        # The user can create multiple notes with the same note form. Each time the 'submit'
        # button is pressed, a new note is added to the notebook.

        b1 = tk.Button(self.frame_main, text='submit', command=self.submit)
        b1.grid(padx=10, pady=10, row=9, column=1, sticky='w')

        b5 = tk.Button(self.frame_main, text='close', command=self.destroy)
        b5.grid(padx=10, pady=10, row=9, column=0) 
       

    
    def submit(self):
        # calculate the date and time information for the meta field
        now = datetime.datetime.now() # gets the current date and time
        local_now = now.astimezone() # shows the local time and the GMT offset
        local_tz = local_now.tzinfo 
        created = datetime.datetime.now()
        
        # get all the input values and put them into a dictionary along with the metadata
        title = self.note_title.get()
        text = self.note_text.get('1.0', 'end').strip('\n')
        link = self.note_link.get()
        tags = self.note_tags.get()
        note_id = self.note_id.get()
        author = self.note_author.get()
        snippet = self.snippet.get('1.0', 'end').strip('\n')
        meta = f'note_id {note_id} created {created}, {local_tz} by {author}'
        note_dict = {'title':title, 'text':text, 'link':link, 'tags':tags, 'note_id':note_id, 'author':author, 'snippet':snippet, 'meta':meta}
        
        # add the dictionary to the notebook
        self.notebook.append(note_dict)
        self.last_id = self.last_id + 1
        self.note_id.delete(0, tk.END)
        self.note_id.insert(0, self.last_id + 1)
        
        return None

# The MakeNote class creates a new note object.
# The MakeNote class is a subclass of the tk.Button class, 
# which means each note instance has tk.Button functionality

class MakeNote(tk.Button):
    def __init__(self, master=None, note_dict=None, notebook=None): # the arguments on this line
        # are inbound, meaning we pass them when we instantiate the object
        super().__init__(master) # on this line we call the __init__ method of tk.Button and pass
        # the master attribute to it. This gives us all the button attributes and functionality
        
        self.notebook = notebook
        
        # define note attributes
        self.title = note_dict['title']
        self.text = note_dict['text']
        self.link = note_dict['link']
        self.tags = note_dict['tags']
        self.note_id = note_dict['note_id']
        self.author = note_dict['author']
        self.snippet = note_dict['snippet']
        self.meta = note_dict['meta']

        # configure note button; this creates a button with two lines of text
        self.config(bg='light gray', text=f"{self.title}\n{self.meta}")
        
        # Bind mouse events
        self.bind("<Enter>", self.on_hover)
        self.bind("<Leave>", self.on_leave)
        self.bind("<Button-1>", self.note_open)

    def save_close(self):
        self.note_window.destroy()
        main_window.save_notebook(notebook=self.notebook)
       

    def on_hover(self, event): # change the background when the cursor hovers over it
        self.config(bg="lightblue")  

    def on_leave(self, event): # change back when not hovering
        self.config(bg="light gray")  # Restore original color
        
    def note_open(self, event): # on mouse click, open note in new top window
        
        # create a new top window
        self.note_window = tk.Toplevel(main_window, bg="light gray", height=600, width=600)
        self.note_window.title(self.title)
        
        # create a frame in the new window that covers the entire window
        self.frame_main = tk.Frame(self.note_window)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light gray')
        
        # create labels in the frame
        title_label = tk.Label(self.frame_main, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')
        title_content = tk.Label(self.frame_main, bg='light gray', text=self.title, wraplength=400, justify=tk.LEFT)
        title_content.grid(padx=10, pady=10, row=1, column=1, sticky='w')        

        text_label = tk.Label(self.frame_main, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')
        text_content = tk.Label(self.frame_main, bg='light gray', text=self.text, wraplength=400, justify=tk.LEFT)
        text_content.grid(padx=10, pady=10, row=2, column=1, sticky='w')

        link_label = tk.Label(self.frame_main, bg='light gray', text='Note Link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')
        link_content = tk.Label(self.frame_main, bg='light gray', text=self.link, wraplength=400, justify=tk.LEFT)
        link_content.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        
        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')
        tag_content = tk.Label(self.frame_main, bg='light gray', text=self.tags, wraplength=400, justify=tk.LEFT)
        tag_content.grid(padx=10, pady=10, row=4, column=1, sticky='w')

        note_id_label = tk.Label(self.frame_main, bg='light gray', text='Note ID:')
        note_id_label.grid(padx=10, pady=10, row=5, column=0, sticky='e')
        note_id_content = tk.Label(self.frame_main, bg='light gray', text=self.note_id, wraplength=400, justify=tk.LEFT)
        note_id_content.grid(padx=10, pady=10, row=5, column=1, sticky='w')

        author_label = tk.Label(self.frame_main, bg='light gray', text='Note Author:')
        author_label.grid(padx=10, pady=10, row=6, column=0, sticky='e')
        author_content = tk.Label(self.frame_main, bg='light gray', text=self.author, wraplength=400, justify=tk.LEFT)
        author_content.grid(padx=10, pady=10, row=6, column=1, sticky='w')

        snippet_label = tk.Label(self.frame_main, bg='light gray', text='Snippet:')
        snippet_label.grid(padx=10, pady=10, row=7, column=0, sticky='e')
        snippet_content = tk.Label(self.frame_main, bg='light gray', text=self.snippet, wraplength=400, justify=tk.LEFT)
        snippet_content.grid(padx=10, pady=10, row=7, column=1, sticky='w')
        
        meta_label = tk.Label(self.frame_main, bg='light gray', text='Note Meta:')
        meta_label.grid(padx=10, pady=10, row=8, column=0, sticky='e')
        meta_content = tk.Label(self.frame_main, bg='light gray', text=self.meta, wraplength=400, justify=tk.LEFT)
        meta_content.grid(padx=10, pady=10, row=8, column=1, sticky='w')        

        # create a button to close the note window
        b10 = tk.Button(self.frame_main, text='save and close', command=self.save_close)
        b10.grid(padx=10, pady=10, row=9, column=0)

        # create a button to edit the note
        b11 = tk.Button(self.frame_main, text='edit note', command=self.note_edit)
        b11.grid(padx=10, pady=10, row=9, column=1)

    def save_edits(self):
        # calculate the date and time information for the meta field
        now = datetime.datetime.now() # gets the current date and time
        local_now = now.astimezone() # shows the local time and the GMT offset
        local_tz = local_now.tzinfo 
        edited = datetime.datetime.now()
        
        # get all the input values and put them into a dictionary along with the metadata
        self.title = self.title_content.get()
        self.text = self.text_content.get('1.0', 'end').strip('\n')
        self.link = self.link_content.get()
        self.tags = self.tag_content.get()
        self.note_id = self.note_id_content.get()
        self.author = self.author_content.get()
        self.snippet = self.snippet_content.get('1.0', 'end').strip('\n')
        
        # modify the metadata to track edits
        self.meta = self.meta + f', edited {edited}, {local_tz}'
        note_dict = {'title':self.title, 'text':self.text, 'link':self.link, 'tags':self.tags, 'note_id':self.note_id, 'author':self.author, 'snippet':self.snippet, 'meta':self.meta}

        new_notebook = []
        for note in self.notebook:
            if note['note_id'] != self.note_id:
                new_notebook.append(note)
            else:
                new_notebook.append(note_dict)
        self.notebook = new_notebook
        
        # destroy the edit window
        self.edit_window.destroy()
        self.note_window.destroy()
        event = None
        self.note_open(event)
        
        
    def note_edit(self):
        # loads note into an editable window
        # create a new top window
        self.edit_window = tk.Toplevel(main_window, bg="light gray", height=700, width=600)
        self.edit_window.title(self.title)


        
        # create a frame in the new window that covers the entire window
        self.frame_main = tk.Frame(self.edit_window)
        self.frame_main.pack(fill=tk.BOTH, expand=True)
        self.frame_main.config(bg='light gray')
        
        # create labels in the frame
        title_label = tk.Label(self.frame_main, bg='light gray', text='Note Title:')
        title_label.grid(padx=10, pady=10, row=1, column=0, sticky='e')
        self.title_content = tk.Entry(self.frame_main, width=80)
        self.title_content.grid(padx=10, pady=10, row=1, column=1, sticky='w')
        self.title_content.insert(0, self.title)

        text_label = tk.Label(self.frame_main, bg='light gray', text='Note Text:')
        text_label.grid(padx=10, pady=10, row=2, column=0, sticky='e')
        self.text_content = tk.Text(self.frame_main, height=8, width=60)
        self.text_content.grid(padx=10, pady=10, row=2, column=1, sticky='w')
        self.text_content.insert('1.0', self.text)

        link_label = tk.Label(self.frame_main, bg='light gray', text='Note Link:')
        link_label.grid(padx=10, pady=10, row=3, column=0, sticky='e')
        self.link_content = tk.Entry(self.frame_main, width=80)
        self.link_content.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        self.link_content.insert(0, self.link)
        
        tag_label = tk.Label(self.frame_main, bg='light gray', text='Note Tags:')
        tag_label.grid(padx=10, pady=10, row=4, column=0, sticky='e')
        self.tag_content = tk.Entry(self.frame_main, width=80)
        self.tag_content.grid(padx=10, pady=10, row=4, column=1, sticky='w')
        self.tag_content.insert(0, self.tags)

        note_id_label = tk.Label(self.frame_main, bg='light gray', text='Note ID:')
        note_id_label.grid(padx=10, pady=10, row=5, column=0, sticky='e')
        self.note_id_content = tk.Entry(self.frame_main, width=80)
        self.note_id_content.grid(padx=10, pady=10, row=5, column=1, sticky='w')
        self.note_id_content.insert(0, self.note_id)

        author_label = tk.Label(self.frame_main, bg='light gray', text='Note Author:')
        author_label.grid(padx=10, pady=10, row=6, column=0, sticky='e')
        self.author_content = tk.Entry(self.frame_main, width=80)
        self.author_content.grid(padx=10, pady=10, row=6, column=1, sticky='w')
        self.author_content.insert(0, self.author)

        snippet_label = tk.Label(self.frame_main, bg='light gray', text='Snippet:')
        snippet_label.grid(padx=10, pady=10, row=7, column=0, sticky='e')
        self.snippet_content = tk.Text(self.frame_main, height=8, width=60)
        self.snippet_content.grid(padx=10, pady=10, row=7, column=1, sticky='w')
        self.snippet_content.insert('1.0', self.snippet)
        
        meta_label = tk.Label(self.frame_main, bg='light gray', text='Note Meta:')
        meta_label.grid(padx=10, pady=10, row=8, column=0, sticky='e')
        self.meta_content = tk.Label(self.frame_main, bg='light gray', text=self.meta, wraplength=400, justify=tk.LEFT)
        self.meta_content.grid(padx=10, pady=10, row=8, column=1, sticky='w')        

        # create a button to close the edit window
        b10 = tk.Button(self.frame_main, text='close', command=self.edit_window.destroy)
        b10.grid(padx=10, pady=10, row=9, column=0)

        # create a button to edit the note
        b11 = tk.Button(self.frame_main, text='save edits', command=self.save_edits)
        b11.grid(padx=10, pady=10, row=9, column=1)
        
        
# main execution

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop()